In [1]:
import math
import re
import sqlite3
import time
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime, timedelta
#import plotly.graph_objects as go
#import plotly.express as px
from bs4 import BeautifulSoup as bs4
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By

from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

meta = MetaData()

# Create an Engine object
engine = create_engine('sqlite:///fantasy_data.db', echo=False)

# Connect to the database
connection = engine.raw_connection()

# Add the custom function to the SQLite database
#connection.create_function("sqrt", 1, math.sqrt)

#conn = sqlite3.connect('fantasy_data.db')
#conn.create_function('sqrt', 1, math.sqrt)

#from optimize_lineup import Optimized_Lineups
from fantasy_utils import Fantasy_Projections as fp
import fantasy_utils as fu
fp = fu.Fantasy_Projections()

In [3]:
def login():
    driver = webdriver.Chrome("C:\\ProgramData\\Anaconda3\\WebDriver\\bin\\chromedriver.exe")
    driver.get('https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin')
    search_box = driver.find_element_by_id('userid')
    search_box.send_keys('gostros09')
    search_box = driver.find_element_by_id('password')
    search_box.send_keys('Segneri9A')
    search_box.submit()
    return driver

In [4]:
def stitch_positions(row):
    pos_code = row[position_priority+['P']]>=5
    return list(pos_code[pos_code].index)

In [5]:
def get_cbs_projections_ros(driver, url):
    #driver.implicitly_wait(10)
    driver.get(url)
    html = driver.page_source
    soup = bs4(html, 'html5lib')
    data = pd.read_html(str(soup.find_all('table')))[0]
    data.columns = data.columns.droplevel()
    data['cbsid'] = data.Action.apply(lambda txt: -1 if len(re.findall(r'\b\d{5,}\b', txt))==0 else re.findall(r'\b\d{5,}\b', txt)[0])
    data['cbsid'] = data.cbsid.astype(int)
    data['owner_id'] = data.Action.apply(lambda txt: re.findall(r'"onTeamId":"(\d+)"', txt)[0] if len(re.findall(r'"onTeamId":"(\d+)"', txt))>0 else -1)
    data = data.iloc[:-1, 1:]
    data['Rank'] = data['Rank'].astype(int)

    try:
        data['Team'] = data.Player.apply(lambda x: x.split()[-1])
    except:
        pass
    try:
        data['CBSNAME'] = data.Player.apply(lambda x: ' '.join(x.split()[:-3]))
    except:
        pass
    data['owner_id'] = data['owner_id'].astype(int)
    try:
        data.drop(columns=['Avail', 'Player', '1B', '2B', '3B', 'CS', 'OBP', 'SLG', 'K'],inplace=True)
        data.rename(columns={'AVG':'BA'}, inplace=True)
    except:
        data.drop(columns=['Avail', 'Player', 'QS', 'CG', 'GS', 'L', 'BS'],inplace=True)
        data.rename(columns={'K':'SO', 'BB':'BBa', 'H':'Ha', 'INNs':'IP'},inplace=True)
        data['Sv+Hld'] = data['S'] + data['HD']
        data['ER'] = data['IP']*data['ERA']/9
        
    data.rename(columns={'CBSNAME':'Player'},inplace=True)
    
    #elig = pd.read_csv('data\\2024-eligibility.csv')
    #data = data.merge(elig, on=['Player', 'cbsid'], how='left')
    
    players = pd.read_sql('SELECT p.Name, p.playerid, p.Pos, pl.cbsid, pl.cbsname FROM players2024 p LEFT JOIN players pl On (p.playerid=pl.IDFANGRAPHS)', conn)
    data = data.merge(players[['cbsid', 'Pos']], on='cbsid', how='left')
    data['Pos'].fillna('',inplace=True)
    return data

In [6]:
def clean_elig(data):
    for pos in ['1B', '2B', '3B', 'SS', 'C', 'OF', 'DH', 'SP', 'RP']:
        data[pos].fillna(0,inplace=True)
        data[pos] = data[pos].astype(int)
    
    try:
        data.loc[(data['Pos'].str.contains('OF')) & (data['OF']<5), 'OF'] = 5
        data.loc[(data['Pos'].str.contains('1B')), '1B'] = 5
        data.loc[(data['Pos'].str.contains('2B')), '2B'] = 5
        data.loc[(data['Pos'].str.contains('3B')), '3B'] = 5
        data.loc[(data['Pos'].str.contains('SS')), 'SS'] = 5
        data.loc[(data['Pos'].str.contains('C')), 'C'] = 5
        data.loc[(data['1B']>=5) | (data['3B']>=5), 'CI'] = 5
        data.loc[(data['2B']>=5) | (data['SS']>=5), 'MI'] = 5
        data.loc[(data['Pos'].str.contains('P')), 'P'] = 5
    except:
        data.loc[(data['Pos'].str.contains('SP')), 'SP'] = 5
        data.loc[(data['Pos'].str.contains('RP')), 'RP'] = 5
        data.loc[(data['Pos'].str.contains('P')), 'P'] = 5
        
    data.loc[(data['SP']>=5) | (data['RP']>=5) | (data['P']>=5), 'type'] = 'p'
    data['type'].fillna('h',inplace=True)
    data.loc[(data['Player']=='Shohei Ohtani'), 'type'] = 'h'
    data.loc[data['type']=='h', 'DH'] = 5
    for pos in ['P', 'DH', 'CI', 'MI']:
        data[pos].fillna(0,inplace=True)
        data[pos] = data[pos].astype(int)
    data['all_pos'] = data.apply(lambda x: stitch_positions(x), axis=1)
    return data

In [54]:
def scrape_cbs_owner_weekly_roster(soup, ids, owner, wk):
    id_dict = {i.text:int(i['href'].split('/')[-1]) for i in soup.find_all("a", {"class": "playerLink"}, href=True) if i.text!=''}
    df = pd.DataFrame()
    for idx in range(2):
        #t = pd.read_html(str(soup.find_all('table')[1:25]))[idx]
        t = pd.read_html(str(soup.find_all('table')))[idx]
        t.columns = t.columns.droplevel().droplevel()
        t = t[['Pos', 'Players', 'Salary']]
        t['Week'] = wk
        t['Owner'] = owner
        
        if idx==0: # for batters
            reserves_idx = t[t['Pos']=='Reserves'].index.values[0]
            t = pd.concat([t.iloc[:reserves_idx], t.iloc[reserves_idx+1:]])
            t.loc[:reserves_idx, 'Decision'] = 'Start'
            t.loc[reserves_idx:, 'Decision'] = 'Sit'
        else: # for pitchers
            reserves_idx = t[t['Pos']=='Reserves'].index.values[0]
            t = pd.concat([t.iloc[:reserves_idx], t.iloc[reserves_idx+1:-1]])
            t.loc[:reserves_idx, 'Decision'] = 'Start'
            t.loc[reserves_idx:t.index.max(), 'Decision'] = 'Sit'
        
        try:
            t['Team'] = t.Players.apply(lambda x: x.split()[-1])
        except:
            pass
        try:
            t['Player'] = t.Players.apply(lambda x: ' '.join(x.split()[:-3]))
        except:
            pass

        t = t.merge(pd.Series(id_dict).to_frame('CBSID'), left_on='Player', right_index=True, how='left')
        t = t.merge(ids[['CBSID', 'IDFANGRAPHS', 'MLBID']], on='CBSID', how='left', indicator=True)[['Player', 'Owner', 'Week', 'Team', 'Pos', 'Decision', 'CBSID', 'MLBID', 'IDFANGRAPHS']]
        df = pd.concat([df,t])
    
    df.rename(columns={'Player':'CBSNAME'},inplace=True)
    return df

In [8]:
def click_week(wk):
    details = driver.find_element_by_xpath("//div[@class ='select_form_div darkFilter']")
    res = details.find_elements_by_xpath("//li")
    for i in res:
        if i.text==wk:
            i.click()
    return

In [68]:
def get_weekly_sit_or_start_by_owner(owner_name, owner_id, ids, current_period):
    driver.get(f"https://xdl.baseball.cbssports.com/teams/{owner_id}")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, 0);")

    data = pd.DataFrame()
    for wk in range(1,current_period):
        if wk==1:
            try:
                driver.find_element_by_xpath(f"//div[@class ='select_form_div darkFilter']/div/span[text()='Off Season']").click()
            except:
                driver.find_element_by_xpath(f"//div[@class ='select_form_div darkFilter']/div/span[text()='{current_period}']").click()
            driver.implicitly_wait(4)
            driver.find_element_by_xpath(f"//div[@class ='select_form_div darkFilter']/ul/li[@value='/teams/roster-report/{owner_id}/1/']").click()
        else:
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(.5)
            driver.find_element_by_xpath(f"//div[@class ='select_form_div darkFilter']/div/span[text()='{wk-1}']").click()
            time.sleep(2)
            elem = driver.find_elements_by_xpath(f"//div[@class ='select_form_div darkFilter']/ul/li")
            driver.implicitly_wait(4)
            time.sleep(2)
            for e in elem:
                if e.text==str(wk):
                    e.click()

        time.sleep(2)
        html = driver.page_source
        soup = bs4(html, 'html.parser')
        data = pd.concat([data, scrape_cbs_owner_weekly_roster(soup, ids, owner_name, wk)])
        data['year'] = datetime.now().year
        data['owner_id'] = owner_id
    
    data.rename(columns={'CBSID':'cbsid', 'Week':'week', 'Pos':'pos'}, inplace=True)
    data.to_excel(f'data//{datetime.now().year}-{owner_name}-sit-start.xlsx', engine='openpyxl', index=False)
    return data

In [10]:
def write_cbs_values_csv():
    import fantasy_utils as fu
    fp = fu.Fantasy_Projections()
    from bs4 import BeautifulSoup as bs4
    from selenium.webdriver import Chrome
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    import time
    driver = login()
    time.sleep(2)
    driver.get(f"https://xdl.baseball.cbssports.com/features/projected-salaries?:sort_col=4&:sort_dir=1")
    time.sleep(2)
    html = driver.page_source
    soup = bs4(html, 'html.parser')
    df = pd.read_html(str(soup.find_all('table')))[0]
    df.columns = df.loc[1]
    df = df.loc[2:]
    df.reset_index(drop=True,inplace=True)
    df.drop(columns=['AL-Only', 'NL-Only'],inplace=True)
    df['CBSNAME'] = df.Name.apply(lambda x: ' '.join(x.split()[:-3]))
    df.rename(columns={'Mixed':'CBS'},inplace=True)
    df = df[['CBSNAME', 'Pos', 'Team', 'CBS']]
    df['CBS'] = df.CBS.replace('-','$0')
    df['CBS'] = df['CBS'].apply(lambda x: int(x[1:]))
    id_dict = {int(i['href'].split('/')[-1]):[i.text, i['aria-label'].split(' ')[-1]] for i in soup.find_all("a", {"class": "playerLink"}, href=True) if i.text!=''}
    id_df = pd.DataFrame(id_dict, index=['CBSNAME', 'Team']).T
    df = df.merge(id_df.reset_index(), on=['CBSNAME', 'Team'], how='left')
    df.rename(columns={'index':'CBSID'},inplace=True)
    df = df[['CBSID', 'CBSNAME', 'Pos', 'Team', 'CBS']]
    ids = fp.load_id_map()
    idx = df.sort_values('CBSNAME').loc[df['CBSID'].isna(),'CBSID'].index
    df.loc[idx, 'CBSID'] = [int(val) for val in ids.sort_values('CBSNAME').loc[ids['CBSNAME'].isin(df.sort_values('CBSNAME').loc[df['CBSID'].isna(),'CBSNAME'].tolist())]['CBSID'].tolist()]
    df = df.merge(ids[['CBSID', 'IDFANGRAPHS']], on='CBSID', how='left')
    df.rename(columns={"IDFANGRAPHS":'playerid'},inplace=True)
    df.to_csv('data/'+str(fp.yr)+'-cbs-values.csv',index=False)
    return df

In [11]:
def login():
    driver = webdriver.Chrome("C:\\ProgramData\\Anaconda3\\WebDriver\\bin\\chromedriver.exe")
    driver.get('https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin')
    search_box = driver.find_element_by_id('userid')
    search_box.send_keys('gostros09')
    search_box = driver.find_element_by_id('password')
    search_box.send_keys('Segneri9A')
    search_box.submit()
    return driver

def get_cbsid_from_eligibility(s):
    player_ids = []

    # Find all the 'a' tags (links) within the table
    links = s.find('table').find_all('a')

    # Iterate over links
    for link in links:
        # Get the href attribute value
        href = link.get('href')
        # Extract the number at the end using regular expressions
        try:
            player_id = re.search(r'/(\d+)$', href).group(1)
            if int(player_id) not in player_ids:
                player_ids.append(int(player_id))
        except:
            pass
    
    return player_ids

def process_eligibility(val, driver):
    driver.get(elig_url+val)
    time.sleep(2.2)
    html = driver.page_source
    soup = bs4(html, 'html.parser')
    elig = pd.read_html(str(soup.find_all('table')))[0]
    elig = elig.iloc[:-1,:]
    player_ids = get_cbsid_from_eligibility(soup)
    elig['cbsid'] = player_ids
    elig['Player'] = elig['Player'].apply(lambda x: x[:re.search(r'RP|SP|1B|2B|3B|SS|OF|LF|CF|RF|DH|C\s', x).span()[0]])
    elig.fillna(0,inplace=True)
    elig.loc[:,1:] = elig.iloc[:,1:].astype('int32')
    return elig

In [9]:
#df = write_cbs_values_csv()

In [12]:
pd.read_sql('owners', engine)#.to_dict(orient='records')

,owner_id,owner
0,30,9 Grand Kids
1,3,Brewbirds
2,36,Charmer
3,41,Dirty Birds
4,4,Harveys Wallbangers
5,27,Lil Trump & the Ivanabees
6,38,Lima Time!
7,44,Roid Ragers
8,1,Trouble with the Curve
9,29,Ugly Spuds


In [13]:
owner_dict = {"9 Grand Kids":'30', 'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', "Harveys Wallbangers":'4', 'Lil Trump & the Ivanabees':'27', 'Lima Time!':'38', 
                        'Roid Ragers':'44', 'Trouble with the Curve':'1', 'Ugly Spuds':'29', 'Wiscompton Wu-Tang':'42', 'Young Guns':'45'}
position_priority = ['C', '2B', '3B', 'SS', 'OF', '1B', 'MI', 'CI', 'DH', 'SP', 'RP']

In [14]:
name_change = {"9 Gran...":'9 Grand Kids', 'Brewbi...':'Brewbirds', 'Charme...':'Charmer', 'Dirty ...':'Dirty Birds', 
               'Harvey...':'Harveys Wallgangers', 'Lil Tr...':'Lil Trump & the Ivanabees', 'Lima T...':'Lima Time!', 
               'Roid R...':'Roid Ragers', 'Troubl...':'Trouble with the Curve', 'Ugly S...':'Ugly Spuds', 
               'Wiscom...':'Wiscompton Wu-Tang', 'Young ...':'Young Guns', "Mom's ...":"Mom's Cookin"}

In [15]:
team_list = ['Lima Time!', 'Ugly Spuds', 'Harveys Wallbangers', 'Roid Ragers', 'Charmer', 'Wiscompton Wu-Tang', '9 Grand Kids', 'Brewbirds', 'Trouble With The Curve', "Mom's Cookin", 'Dirty Birds', 'Young Guns']
stat_list = ['BA', 'HR', 'R', 'RBI', 'SB', 'ERA', 'Sv+Hld', 'SO', 'W', 'WHIP']

In [16]:
opening_week = int(datetime.strftime(pd.Timestamp('2024-03-28'), '%W')) - 1
cur_week = int(datetime.strftime(datetime.now(), '%W'))
period = cur_week - opening_week
current_period = period + 1
opening_week, cur_week, period, current_period

(12, 20, 8, 9)

In [17]:
h_proj_url = 'https://xdl.baseball.cbssports.com/stats/stats-main/all:C:1B:2B:3B:SS:MI:CI:OF:DH/restofseason:p/standard/projections?print_rows=9999'
h_proj_cur_url = f'https://xdl.baseball.cbssports.com/stats/stats-main/all:C:1B:2B:3B:SS:MI:CI:OF:DH/{current_period}/standard/projections?print_rows=9999'

p_proj_url = 'https://xdl.baseball.cbssports.com/stats/stats-main/all:SP:RP:P/restofseason:p/standard/projections?print_rows=9999'
p_proj_cur_url = f'https://xdl.baseball.cbssports.com/stats/stats-main/all:SP:RP/{current_period}/standard/projections?print_rows=9999'

eligibility_url_dict = {"9 Grand Kids":'30', 'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', "Harveys Wallbangers":'4', 'Lil Trump & the Ivanabees':'27', 'Lima Time!':'38', 
                        'Roid Ragers':'44', 'Trouble with the Curve':'1', 'Ugly Spuds':'29', 'Wiscompton Wu-Tang':'42', 'Young Guns':'45', "Mom's Cookin":'47'}

https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json

In [18]:
driver = login()

In [19]:
elig_url = 'https://xdl.baseball.cbssports.com/teams/eligibility/'
elig = pd.DataFrame()

for key, val in eligibility_url_dict.items():
    print(key)
    try:
        elig = pd.concat([elig, process_eligibility(val, driver)])
    except:
        print('...skipped')



9 Grand Kids


C:\Users\pddnh\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:719: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  indexer = self._get_setitem_indexer(key)


Brewbirds
Charmer
Dirty Birds
Harveys Wallbangers
Lil Trump & the Ivanabees
...skipped
Lima Time!
Roid Ragers
Trouble with the Curve
Ugly Spuds
Wiscompton Wu-Tang
Young Guns
Mom's Cookin


In [20]:
elig.to_csv('data\\2024-eligibility-period-'+str(current_period)+'.csv', index=False)

In [21]:
elig0 = elig.copy()
elig

,Player,C,1B,2B,3B,SS,MI,CI,OF,DH,SP,RP,cbsid
0,Trea Turner,0,0,0,0,32,32,0,0,1,0,0,2135249
1,Seth Lugo,0,0,0,0,0,0,0,0,0,9,0,2210221
2,Ian Happ,0,0,0,0,0,0,0,33,5,0,0,2184356
3,J.T. Realmuto,34,0,0,0,0,0,0,0,0,0,0,1947839
4,Carlos Rodon,0,0,0,0,0,0,0,0,0,8,0,2138665
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,Max Muncy,0,0,0,37,0,0,37,0,0,0,0,2106304
29,John Means,0,0,0,0,0,0,0,0,0,2,0,2824299
30,Frankie Montas,0,0,0,0,0,0,0,0,0,7,0,2117122
31,Jose Alvarado,0,0,0,0,0,0,0,0,0,0,18,2444515


In [22]:
players = pd.read_sql('SELECT p.Name, p.playerid, p.Pos, pl.cbsid, pl.cbsname FROM players2024 p LEFT JOIN players pl On (p.playerid=pl.IDFANGRAPHS)', conn)

In [23]:
players.columns

Index(['Name', 'playerid', 'Pos', 'cbsid', 'CBSNAME'], dtype='object')

In [41]:
#data = get_cbs_projections_ros(driver, h_proj_url)
data = get_cbs_projections_ros(driver, h_proj_cur_url)

print(data.shape)
data = data.merge(elig, on=['Player', 'cbsid'], how='left')
data = clean_elig(data)

(2985, 14)


In [42]:
#pdata = get_cbs_projections_ros(driver, p_proj_url)
pdata = get_cbs_projections_ros(driver, p_proj_cur_url)

print(pdata.shape)
pdata = pdata.merge(elig, on=['Player', 'cbsid'], how='left')
pdata = clean_elig(pdata)

(2499, 18)


In [43]:
df = pd.concat([data,pdata[pdata['Player']!='Shohei Ohtani']])
try:
    df.drop(columns='owner_id_y',inplace=True)
    df.rename(columns={'owner_id_x':'owner_id'}, inplace=True)
except:
    pass

In [44]:
h_mask = (df['owner_id']>=0) & (df['AB']>0) #(df['Pos'].isin(['C', '1B', '2B', '3B', 'LF', 'CF', 'RF', 'SS', 'DH']))
hitters = (df['type']=='h') #(df['Pos'].isin(['C', '1B', '2B', '3B', 'LF', 'CF', 'RF', 'SS', 'DH']))
p_mask = (df['owner_id']>=0) & (df['IP']>0) #(df['Pos'].isin(['SP', 'RP']))
pitchers = (df['type']=='p') #(df['Pos'].isin(['SP', 'RP']))

lgBA = df[h_mask]['H'].sum()/df[h_mask]['AB'].sum()
df.loc[hitters, 'zlgBA'] = (df[hitters]['H'] - (df[hitters]['AB'] * lgBA))

lgERA = df[p_mask]['ER'].sum()/df[p_mask]['IP'].sum()*9
df.loc[pitchers, 'zlgERA'] = (((df[pitchers]['ER']*9) - (df[pitchers]['IP']*lgERA))*-1)

lgWHIP = (df[p_mask]['BBa'].sum()+df[p_mask]['Ha'].sum())/df[p_mask]['IP'].sum()
df.loc[pitchers, 'zlgWHIP'] = (((df.loc[pitchers]['Ha']+df.loc[pitchers]['BBa'])-(df.loc[pitchers]['IP']*lgWHIP))*-1)

q = df[h_mask][['HR', 'RBI', 'R', 'SB', 'H', 'AB']].describe().T[['mean', 'std']].T.to_dict()
q['BA'] = {'mean':df[h_mask]['H'].sum()/df[h_mask]['AB'].sum(), 'std':df[h_mask]['BA'].std()}
q.update(df[p_mask][['W', 'SO', 'Sv+Hld', 'IP', 'ER', 'BBa', 'Ha']].describe().T[['mean', 'std']].T.to_dict())
q['ERA'] = {'mean':(df[p_mask]['ER'].sum()/df[p_mask]['IP'].sum())*9, 'std':(df[p_mask]['ER']/df[p_mask]['IP']*9).std()}
q['WHIP'] = {'mean':(df[p_mask]['BBa'].sum()+df[p_mask]['Ha'].sum())/df[p_mask]['IP'].sum(), 'std':((df[p_mask]['BBa']+df[p_mask]['Ha'])/df[p_mask]['IP']).std()}
q['zlgBA'] = {'mean':df[h_mask]['zlgBA'].mean(), 'std':df[h_mask]['zlgBA'].std()}
q['zlgERA'] = {'mean':df[p_mask]['zlgERA'].mean(), 'std':df[p_mask]['zlgERA'].std()}
q['zlgWHIP'] = {'mean':df[p_mask]['zlgWHIP'].mean(), 'std':df[p_mask]['zlgWHIP'].std()}
q

{'HR': {'mean': 0.7357843137254898, 'std': 0.34347381074688194},
 'RBI': {'mean': 2.642647058823529, 'std': 0.8494535206153949},
 'R': {'mean': 2.675490196078433, 'std': 0.8280710893199013},
 'SB': {'mean': 0.4348039215686271, 'std': 0.4290147502652289},
 'H': {'mean': 4.8509803921568615, 'std': 1.2827575820832104},
 'AB': {'mean': 17.956862745098046, 'std': 3.8960763338007496},
 'BA': {'mean': 0.2701463201572396, 'std': 0.024992354669865827},
 'W': {'mean': 0.31689189189189226, 'std': 0.21072873671625547},
 'SO': {'mean': 5.637162162162161, 'std': 2.7853425980461606},
 'Sv+Hld': {'mean': 0.3533783783783784, 'std': 0.5782233568558767},
 'IP': {'mean': 5.247972972972982, 'std': 3.0094515421083945},
 'ER': {'mean': 2.1121126126126133, 'std': 1.2916280095637802},
 'BBa': {'mean': 1.7033783783783787, 'std': 0.8866673474914684},
 'Ha': {'mean': 4.978378378378381, 'std': 2.8653582017896704},
 'ERA': {'mean': 3.6221629972962526, 'std': 0.7691404259639426},
 'WHIP': {'mean': 1.2732071584910518

In [45]:
for stat in ['R', 'HR', 'RBI', 'SB', 'zlgBA']:
    df.loc[hitters, 'z'+stat] = df[hitters].apply(lambda row: fp.big_board(row, stat, q), axis=1)

df.loc[hitters & (df['zlgBA'].isna()), 'zlgBA'] = df['zlgBA'].min()-.01
df.loc[hitters & (df['zzlgBA'].isna()), 'zzlgBA'] = df['zzlgBA'].min()-.01
df.loc[hitters, 'BIGAAh'] = df[hitters]['zR'] + df[hitters]['zRBI'] + df[hitters]['zHR'] + df[hitters]['zSB'] + df[hitters]['zzlgBA']

for stat in ['W', 'SO', 'Sv+Hld', 'zlgERA', 'zlgWHIP']:
    df.loc[pitchers, 'z'+stat] = df[pitchers].apply(lambda row: fp.big_board(row, stat, q), axis=1)

df.loc[pitchers & (df['zzlgERA'].isna()), 'zzlgERA'] = df['zzlgERA'].min()-.01
df.loc[pitchers & (df['zzlgWHIP'].isna()), 'zzlgWHIP'] = df['zzlgWHIP'].min()-.01
df.loc[pitchers, 'BIGAAp'] = df[pitchers]['zW']+df[pitchers]['zSO']+df[pitchers]['zSv+Hld']+df[pitchers]['zzlgERA']+df[pitchers]['zzlgWHIP']

In [46]:
df['BIGAAh'].fillna(0,inplace=True)
df['BIGAAp'].fillna(0,inplace=True)
df['z'] = df['BIGAAh']+df['BIGAAp']

In [47]:
df['year'] = datetime.now().year
df['week'] = current_period
#df['proj_type'] = 'ros'
df['proj_type'] = 'current_period'
df['all_pos'] = df.all_pos.apply(lambda x: ','.join(x))

In [31]:
period, current_period

(8, 9)

In [48]:
#df[df['owner_id']>0].to_csv(f'data\\{datetime.now().year}-period-{current_period}-ros-projections.csv',index=False)
df[df['owner_id']>0].to_csv(f'data\\{datetime.now().year}-period-{current_period}-projections.csv',index=False)

In [33]:
f = pd.read_sql("SELECT * FROM eligibility WHERE year=2024", conn)
f.head()

,cbsid,year,week,all_pos,posC,pos1B,pos2B,pos3B,posSS,posMI,posCI,posOF,posDH,posSP,posRP,posP
0,2211777,2024,1,"OF,DH",0,0,0,0,0,0,0,5,5,0,0,0
1,2901324,2024,1,"DH,P",0,0,0,0,0,0,0,0,5,0,0,5
2,2071264,2024,1,"OF,DH",0,0,0,0,0,0,0,5,5,0,0,0
3,2106654,2024,1,"2B,SS,OF,MI,DH",0,0,5,0,5,5,0,5,5,0,0,0
4,3117913,2024,1,"OF,DH",0,0,0,0,0,0,0,5,5,0,0,0


In [34]:
elig = elig.merge(df[['cbsid', 'all_pos', 'owner_id']], on='cbsid', how='left')
elig.loc[(elig['SP']>0), 'P'] = 5
elig.loc[(elig['RP']>0), 'P'] = 5
elig['P'].fillna(0,inplace=True)
elig['year'] = datetime.now().year
elig['week'] = current_period

elig

,Player,C,1B,2B,3B,SS,MI,CI,OF,DH,SP,RP,cbsid,all_pos,owner_id,P,year,week
0,Trea Turner,0,0,0,0,32,32,0,0,1,0,0,2135249,"SS,MI,DH",30.0,0.0,2024,9
1,Seth Lugo,0,0,0,0,0,0,0,0,0,9,0,2210221,"SP,P",30.0,5.0,2024,9
2,Ian Happ,0,0,0,0,0,0,0,33,5,0,0,2184356,"OF,DH",30.0,0.0,2024,9
3,J.T. Realmuto,34,0,0,0,0,0,0,0,0,0,0,1947839,"C,DH",30.0,0.0,2024,9
4,Carlos Rodon,0,0,0,0,0,0,0,0,0,8,0,2138665,"SP,P",30.0,5.0,2024,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,Max Muncy,0,0,0,37,0,0,37,0,0,0,0,2106304,"3B,CI,DH",47.0,0.0,2024,9
392,John Means,0,0,0,0,0,0,0,0,0,2,0,2824299,P,47.0,5.0,2024,9
393,Frankie Montas,0,0,0,0,0,0,0,0,0,7,0,2117122,"SP,P",47.0,5.0,2024,9
394,Jose Alvarado,0,0,0,0,0,0,0,0,0,0,18,2444515,"RP,P",47.0,5.0,2024,9


In [35]:
for i, el in elig[elig['all_pos'].notna()].iterrows():
    for p in el['all_pos'].split(','):
        elig.at[i,str(p)] = 5

In [36]:
e = elig.copy()#df.copy()
for col in e:
    if col in ['C', '1B', '2B', '3B', 'SS', 'OF', 'MI', 'CI', 'DH', 'SP', 'RP', 'P']:
        e.rename(columns={col:'pos'+col},inplace=True)

In [37]:
e[e['owner_id']>-1][['cbsid', 'year', 'week', 'all_pos', 'posC', 'pos1B', 'pos2B', 'pos3B', 'posSS', 'posMI', 'posCI', 'posOF', 'posDH', 'posSP', 'posRP', 'posP']]

,cbsid,year,week,all_pos,posC,pos1B,pos2B,pos3B,posSS,posMI,posCI,posOF,posDH,posSP,posRP,posP
0,2135249,2024,9,"SS,MI,DH",0,0,0,0,5,5,0,0,5,0,0,0.0
1,2210221,2024,9,"SP,P",0,0,0,0,0,0,0,0,0,5,0,5.0
2,2184356,2024,9,"OF,DH",0,0,0,0,0,0,0,5,5,0,0,0.0
3,1947839,2024,9,"C,DH",5,0,0,0,0,0,0,0,5,0,0,0.0
4,2138665,2024,9,"SP,P",0,0,0,0,0,0,0,0,0,5,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,2106304,2024,9,"3B,CI,DH",0,0,0,5,0,0,5,0,5,0,0,0.0
392,2824299,2024,9,P,0,0,0,0,0,0,0,0,0,2,0,5.0
393,2117122,2024,9,"SP,P",0,0,0,0,0,0,0,0,0,5,0,5.0
394,2444515,2024,9,"RP,P",0,0,0,0,0,0,0,0,0,0,5,5.0


In [38]:
# Appends to eligibility table
e[e['owner_id']>-1][['cbsid', 'year', 'week', 'all_pos', 'posC', 'pos1B', 'pos2B', 'pos3B', 'posSS', 'posMI', 'posCI', 'posOF', 'posDH', 'posSP', 'posRP', 'posP']]\
    .to_sql('eligibility', con=conn, if_exists='append', index=False)

In [39]:
# Appends to projections table
df[(df['owner_id']>-1) | (df['Rank']<500)][['cbsid', 'year', 'week', 'proj_type', 'AB', 'R', 'H', 'HR', 'RBI', 'BB', 'SB', 'BA', 'IP', 'W', 'SO', 'Sv+Hld', 'BBa', 'Ha', 'ERA', 'WHIP', 'ER', 'z']]\
    .to_sql('projections', con=conn, if_exists='append', index=False)

In [95]:
pd.read_sql("SELECT * FROM projections WHERE cbsid=2211777", conn)

,cbsid,year,week,proj_type,AB,R,H,HR,RBI,BB,SB,BA,IP,W,SO,Sv+Hld,BBa,Ha,ERA,WHIP,ER,z
0,2211777,2024,1,ros,578.0,130.0,191.0,33.0,95.0,69.0,60.0,0.330,None,None,None,None,None,None,None,None,None,14.436478
1,2211777,2024,2,ros,564.0,129.0,188.0,31.0,91.0,68.0,57.0,0.333,None,None,None,None,None,None,None,None,None,14.070892
2,2211777,2024,3,ros,550.0,123.0,177.0,28.0,87.0,66.0,51.0,0.322,None,None,None,None,None,None,None,None,None,13.099049
3,2211777,2024,4,ros,524.0,122.0,169.0,25.0,80.0,67.0,56.0,0.323,None,None,None,None,None,None,None,None,None,13.628484
4,2211777,2024,6,ros,502.0,111.0,160.0,22.0,71.0,63.0,50.0,0.319,None,None,None,None,None,None,None,None,None,12.200464
5,2211777,2024,7,ros,481.0,110.0,148.0,21.0,65.0,63.0,48.0,0.308,None,None,None,None,None,None,None,None,None,11.618636
6,2211777,2024,8,ros,465.0,102.0,143.0,19.0,59.0,60.0,47.0,0.307,None,None,None,None,None,None,None,None,None,11.536377
7,2211777,2024,9,ros,447.0,94.0,134.0,19.0,56.0,59.0,43.0,0.300,None,None,None,None,None,None,None,None,None,10.581659


### Weekly Sit or Start Decisions

In [74]:
current_period

9

In [ ]:
ids = fp.load_id_map()

In [92]:
for key, val in eligibility_url_dict.items():
    print(val, key)
    try:
        data = get_weekly_sit_or_start_by_owner(key, val, ids, current_period)
        if data.shape[0] > 0:
            data[['cbsid', 'owner_id', 'year', 'week', 'pos']].to_sql('roster', con=conn, if_exists='append', index=False)
    except:
        print(f"There was an error processing {val}")

30 9 Grand Kids
3 Brewbirds
36 Charmer
41 Dirty Birds
4 Harveys Wallbangers
27 Lil Trump & the Ivanabees
There was an error processing 27
38 Lima Time!
44 Roid Ragers
1 Trouble with the Curve
29 Ugly Spuds
42 Wiscompton Wu-Tang
45 Young Guns
47 Mom's Cookin


In [93]:
pd.read_sql("SELECT * FROM roster WHERE year=2024", conn)

,cbsid,owner_id,year,week,pos
0,1947839,30,2024,1,C
1,1793168,30,2024,1,1B
2,2167464,30,2024,1,2B
3,2051255,30,2024,1,3B
4,2135249,30,2024,1,SS
...,...,...,...,...,...
3171,2050483,47,2024,8,SP
3172,2042426,47,2024,8,SP
3173,2449982,47,2024,8,RP
3174,2117122,47,2024,8,RP


In [90]:
# Delete statement

# Create an engine
engine = create_engine('sqlite:///fantasy_data.db', echo=False)

# Using a connection with a transaction context manager
with engine.connect() as connection:
    # Begin a transaction explicitly
    with connection.begin() as transaction:
        delete_statement = text("DELETE FROM roster WHERE year=:year")
        #result = connection.execute(delete_statement, {'year': 2024})
        # Transaction is committed or rolled back automatically at the end of the context block


In [94]:
html('hr')

NameError: name 'html' is not defined

In [29]:
def fill_pos(row):
    for pos in row['Pos'].split('/'):
        print(pos)
        data.loc[data['cbsid']==row['cbsid'], pos] = 5
    return row

In [ ]:
html = driver.page_source
soup = bs4(html, 'html5lib')
data = pd.read_html(str(soup.find_all('table')))[0]
data.columns = data.columns.droplevel()
data = data.iloc[:-1, 1:]
data['Rank'] = data['Rank'].astype(int)

try:
    data['Team'] = data.Player.apply(lambda x: x.split()[-1])
except:
    pass
try:
    data['CBSNAME'] = data.Player.apply(lambda x: ' '.join(x.split()[:-3]))
except:
    pass

id_dict = {int(i['href'].split('/')[-1]):[i.text, i['aria-label'].split(' ')[-1]] for i in soup.find_all("a", {"class": "playerLink"}, href=True) if i.text!=''}
id_df = pd.DataFrame(id_dict, index=['CBSNAME', 'Team']).T

data = data.merge(id_df.reset_index(), on=['CBSNAME', 'Team'], how='left')
data.rename(columns={'index':'CBSID'},inplace=True)
data.drop(columns=['Avail'],inplace=True)

for i, row in ids.loc[ids['CBSNAME'].isin(data.loc[data['CBSID'].isna()]['CBSNAME'].tolist())][['CBSNAME', 'CBSID']].iterrows():
    data.loc[data['CBSNAME']==row['CBSNAME'],'CBSID'] = row['CBSID']

data = data.drop_duplicates()

In [ ]:
data = data.merge(ids[ids['CBSID'].notna()][['CBSID', 'IDFANGRAPHS', 'FANGRAPHSNAME']], on='CBSID', how='left')
data = data.drop_duplicates()
data = data.merge(ids[['CBSNAME', 'FANGRAPHSNAME']], on='CBSNAME', how='left', suffixes=['','_cbsname_merge'])
data = data.drop_duplicates()
data['FANGRAPHSNAME'].fillna(data['FANGRAPHSNAME_cbsname_merge'],inplace=True)
data.drop(columns='FANGRAPHSNAME_cbsname_merge',inplace=True)

In [ ]:
data.shape

In [ ]:
data.to_csv('data/2024-cbs-proj-h.csv', index=False)

In [ ]:
driver.implicitly_wait(10)
driver.get('https://xdl.baseball.cbssports.com/stats/stats-main')
time.sleep(4)
driver.find_element_by_xpath(f"//a[@href ='/SP:RP']").click()
time.sleep(1.5)
driver.find_element_by_xpath(f"//a[@href ='/all']").click()
time.sleep(1.5)
driver.execute_script("window.scrollTo(0, 150);")
driver.find_element_by_xpath(f"//div[@class ='select_div_label_container selected_arrow']").click()
time.sleep(2)
elem = driver.find_elements_by_xpath(f"//ul[@class='form_select_ul combinedFilterUl3Cols']/li")


### Begin loop code

In [ ]:
# pitching
#driver.get(f'https://xdl.baseball.cbssports.com/stats/stats-main/all:SP:RP/period-{1}:p/standard/stats?print_rows=999')
# hitting
driver.get(f'https://xdl.baseball.cbssports.com/stats/stats-main/all:C:1B:2B:3B:SS:MI:CI:OF:DH/period-1:p/standard/stats?print_rows=9999')
# Then need to click "All"

In [ ]:
wk = 0
df = pd.DataFrame()

In [ ]:
wk

In [ ]:
wk += 1
html = driver.page_source
soup = bs4(html, 'html.parser')
data = pd.read_html(str(soup.find_all('table')[1:25]))[0]
data.columns = data.columns.droplevel()
data = data.iloc[:-1, 1:]
data['Rank'] = data['Rank'].astype(int)
data = data[(data['Avail']!='FA') | (data['Rank']<9999)]
data['Week'] = wk

try:
    data['Team'] = data.Player.apply(lambda x: x.split()[-1])
except:
    pass
try:
    data['Player'] = data.Player.apply(lambda x: ' '.join(x.split()[:-3]))
except:
    pass

id_dict = {int(i['href'].split('/')[-1]):[i.text, i['aria-label'].split(' ')[-1]] for i in soup.find_all("a", {"class": "playerLink"}, href=True) if i.text!=''}
id_df = pd.DataFrame(id_dict, index=['CBSNAME', 'Team']).T
data = data.merge(id_df.reset_index(), left_on=['Player', 'Team'], right_on=['CBSNAME', 'Team'], how='left')

df = pd.concat([df,data])
time.sleep(2)
if wk<27:
    driver.get(f'https://xdl.baseball.cbssports.com/stats/stats-main/all:C:1B:2B:3B:SS:MI:CI:OF:DH/period-{wk+1}:p/standard/stats?print_rows=9999')

In [ ]:
df.shape

In [ ]:
data

### End looping code

In [ ]:
df.rename(columns={'index':'cbsid'},inplace=True)
df = df.replace(name_change)
df.loc[df['Player']=='Logan O\'Hoppe', 'cbsid'] = 3166980
df.loc[df['Player']=='Travis d\'Arnaud', 'cbsid'] = 1730742
df.loc[df['Player']=='Brian O\'Keefe', 'cbsid'] = 3089588
df.loc[df['Player']=='Ke\'Bryan Hayes', 'cbsid'] = 2211187
df.loc[df['Player']=='Tyler O\'Neill', 'cbsid'] = 2171342
df.loc[df['Player']=='Ryan O\'Hearn', 'cbsid'] = 2211177
#df = df.merge(ids[['CBSID', 'IDFANGRAPHS', 'MLBID']], left_on='cbsid', right_on='CBSID', how='left', indicator=True)

In [ ]:
df = pd.read_excel('data//2023-weekly-stats-h.xlsx')

In [ ]:
df[df['cbsid'].isna()].Player.unique()

In [ ]:
#df.to_excel('data//2023-weekly-holds.xlsx')
#dfp = df.copy()

In [ ]:
dfh = df.copy()
#dfh.to_excel('data//2023-weekly-stats-new.xlsx')

In [ ]:
dfh.shape

In [ ]:
dfh = pd.read_excel('data//2023-weekly-stats-new.xlsx')
dfh.fillna(0,inplace=True)

In [ ]:
dfh['year'] = 2023
dfh.rename(columns={'Week':'week'},inplace=True)

In [ ]:
owners = pd.read_sql('roster', engine)
owners.shape

In [ ]:
owners.info()

In [ ]:
dfh.info()

In [ ]:
pd.pivot_table(owners, values='cbsid', aggfunc='count', index='owner_id', columns='week')#.sum().sum()

In [ ]:
pd.pivot_table(dfh, columns='week', values='cbsid', aggfunc='count')

In [ ]:
dfh[['cbsid', 'year', 'week', 'R', 'RBI', 'HR', 'SB', 'H', 'AB']].merge(owners.query('pos!="B" & pos!="SP"'), on=['cbsid', 'year', 'week'], how='inner').query('owner_id==36 & week==1')

In [ ]:
pd.pivot_table(dfh[['cbsid', 'year', 'week', 'R', 'RBI', 'HR', 'SB', 'H', 'AB']].merge(owners, on=['cbsid', 'year', 'week'], how='inner').query('pos!="B" & pos!="SP" & pos!="RP" & pos!="P"'),
               index='owner_id', columns='week', values='HR', aggfunc='sum')

In [ ]:
p = pd.read_excel('data//2023-weekly-stats-p.xlsx')
#p['year'] = 2023
#p.rename(columns={'Outs':'outs', 'Week':'week'},inplace=True)
#p.rename(columns={'H':'Ha', 'BB':'BBa', 'K':'SO'},inplace=True)

In [ ]:
p[p['cbsid'].isna()]

In [ ]:
#p['outs'] = p.INNs.apply(lambda x: (int(str(x).split('.')[0])*3) + int(str(x).split('.')[1]))
#p['IP'] = round(p['outs']/3.,2)
#p['ER'] = round(p['ERA']*p['IP']/9.,1)
#p['SvHld'] = p['S']+p['HD']

In [ ]:
p.to_excel('data//2023-weekly-stats-p.xlsx')

In [ ]:
pd.pivot_table(p, columns='week', values='cbsid', aggfunc='count').reset_index().sort_values(1, ascending=False)

In [ ]:
pd.pivot_table(p[['cbsid', 'year', 'week', 'W', 'SO', 'SvHld', 'ER', 'Ha', 'BBa', 'IP', 'outs']].merge(owners, on=['cbsid', 'year', 'week'], how='inner').query('pos=="SP" | pos=="RP" | pos=="P"'),
               index='owner_id', columns='week', values='SO', aggfunc='sum').sort_values(1,ascending=False)

In [ ]:
p[['cbsid', 'year', 'week', 'W', 'SO', 'SvHld', 'ER', 'Ha', 'BBa', 'IP', 'outs']].merge(owners, on=['cbsid', 'year', 'week'], how='inner').query('cbsid==2901324')

In [ ]:
dfh[['cbsid', 'year', 'week', 'R', 'RBI', 'HR', 'SB', 'H', 'AB']].merge(owners, on=['cbsid', 'year', 'week'], how='inner').query('cbsid==2901324')

In [ ]:
#dfh = dfh[['cbsid', 'year', 'week', 'R', 'RBI', 'HR', 'SB', 'H', 'AB']].sort_values(['cbsid', 'year', 'week'])
p = p[['cbsid', 'year', 'week',  'W', 'SO', 'SvHld', 'ER', 'Ha', 'BBa', 'IP', 'outs']].sort_values(['cbsid', 'year', 'week'])

In [ ]:
stats = dfh.merge(p, on=['cbsid', 'year', 'week'],how='outer')#.fillna(0)

In [ ]:
#for col in ['cbsid', 'R', 'RBI', 'HR', 'SB', 'H', 'AB', 'W', 'SO', 'SV+HLD', 'ER', 'Ha', 'BBa', 'outs']:
#    stats[col] = stats[col].astype(int)

In [ ]:
# Uncomment to append to stats table
# ! make sure not to fill nan values !
#stats.sort_values(['cbsid', 'year', 'week']).to_sql('stats', if_exists='replace', con=engine, index=False)

#### need a way to remove data where the player did not hit or did not pitch, or need to make two separate stats tables

In [ ]:
pd.pivot_table(stats, index='week', values=['R', 'RBI', 'HR', 'SB'], aggfunc=['mean', 'std'])

In [ ]:
pd.read_sql("SELECT name FROM sqlite_master", engine)